In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import requests
import datetime as dt # 時間套件
from bs4 import BeautifulSoup
from openai import OpenAI ,OpenAIError
def stock_fundamental(stock_id):
    stock = yf.Ticker(stock_id)
     # 營收成長率
    try:
        quarterly_revenue_growth = np.round(stock.quarterly_financials.loc["Total Revenue"].pct_change(-1).tolist(), 2)[:4]
        # 每季EPS
        quarterly_eps = np.round(stock.quarterly_financials.loc["Basic EPS"].tolist(), 2)[:4]
        # EPS季增率
        quarterly_eps_growth = np.round(stock.quarterly_financials.loc["Basic EPS"].pct_change(-1).tolist(), 2)[:4]
        date = stock.quarterly_financials.columns[:4]
        df = pd.DataFrame({
            "每季日期" : date,
            "營收成長率" : quarterly_revenue_growth,
            "每季EPS" : quarterly_eps ,
            "EPS季增率" : quarterly_eps_growth
    })
        return df
    except:
        dfnan = pd.DataFrame({
            "每季日期" : ['nan','nan','nan','nan'],
            "營收成長率" : ['nan','nan','nan','nan'],
            "每季EPS" : ['nan','nan','nan','nan'],
            "EPS季增率" :['nan','nan','nan','nan']
        
        })
        return dfnan

def news(stock_id):
    stock_id = stock_id[:4]
    data=[]
    datas = requests.get(f'https://ess.api.cnyes.com/ess/api/v1/news/keyword?q={stock_id}&limit=10&page=1').json()['data']['items']
    for item in datas:
        title = item["title"]
        publish_at = item["publishAt"]
        utc_time = dt.datetime.utcfromtimestamp(publish_at)
        formatted_date = utc_time.strftime('%Y-%m-%d')
        data.append([formatted_date ,title])
    return data
def www(stock_id,num):
    stock_id = stock_id[:4]
    datas = requests.get(f'https://ess.api.cnyes.com/ess/api/v1/news/keyword?q={stock_id}&limit=10&page=1').json()['data']['items'][num]['newsId']
    return datas
def get_reply(messages):
    ###############################client = OpenAI(api_key = )################################請輸入openAI密碼
    try:
        response = client.chat.completions.create(
            model = "gpt-4o",
            messages = messages
        )
        reply = response.choices[0].message.content
    except OpenAIError as err:
        reply = f"發生 {err.type} 錯誤\n{err.message}"
    return reply

# 建立訊息指令(Prompt)
def generate_content_msg(stock_id):
    news_data = news(stock_id)
    content_msg = f'請依據以下資料來進行分析並給出一份完整的分析報告:\n'
    stock_value_data = stock_fundamental(stock_id)
    content_msg += f'每季營收資訊：\n {stock_value_data}\n'
    content_msg += f'近期新聞資訊: \n {news_data}\n'
    content_msg += f' 做出近期的趨勢報告,請以詳細、\
      嚴謹及專業的角度撰寫此報告,並提及重要的數字, reply in 繁體中文'
    print(content_msg)
    return content_msg
#做出近期的趨勢報告,請以詳細、\
      #嚴謹及專業的角度撰寫此報告,並提及重要的數字, reply in 繁體中文'
# StockGPT
def stock_gpt(stock_id,type1):
    content_msg = generate_content_msg(stock_id)
    msg = [{
        "role": "system",
        "content": f"{type1}"
    }, {
        "role": "user",
        "content": content_msg
    }]

    reply_data = get_reply(msg)
    
    return reply_data
def id(stock_id) :
    try :
    # 讀取 CSV 檔案
        stock = pd.read_csv('stockid.csv')

# 將公司代號欄位轉換為字串型態
        stock.index = stock['代號']
        stock = stock.dropna(axis=1,how='all')
        df = stock.drop(columns=['代號'])
        return df['公司'][str(stock_id)]
    except:
        return ''

In [2]:
from dash import Dash,html,Output,Input,State,dcc,dash_table
import  yfinance as yf
import plotly.graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
import pandas as pd
from openai import OpenAI ,OpenAIError
import dash_ag_grid as dag
import jieba as ba
import re
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
client = OpenAI(api_key ="") #請輸入密碼
df = yf.download("2330.TW",period="5d")
df=df.reset_index()
stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]
app = Dash(__name__, external_stylesheets=stylesheets)


app.layout = html.Div([
   
    html.Div([
        html.Div([
            html.Div([
                html.H1(children = "　台股數據分析整合平台 STOCK ANALYSIS　　　　　　　　　　　　　　")],className="color"),
                html.Div([
                    html.Img(src='/assets/openai4.png'),
                    html.Img(src='/assets/pythonlogo.png')],style={"low": "65%"}),   
                
        ],className="banner",style={"display": "flex", "flex-direction": "row"}),
        html.Div([ 
            html.Div([
                html.Div([
                html.H1(children = "個股研究", style={"textAlign": "right"})],className="color"),
                html.Img(src='/assets/sot.png'),
             ],style={"display": "flex", "flex-direction": "row","position": "relative", "left": "48%"}),
        ],className="banner"),
         html.Div([
            html.Div([
                html.Label("請輸入股票代號　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　系統資訊僅供參考，投資人若依此為買賣依據，須自負盈虧之責。."),
            ],style={"display": "flex", "flex-direction": "row","position": "relative"}),
             html.Div([
                dcc.Input(id="in",type="text"),
                html.Button(id="button",n_clicks=0,children="Submit"),
                dcc.Loading(type="default",children=html.Div(id="loading-output-1")),
                html.H4('　　　　　　　　　　　你正在查詢：') ,
                html.H4(id='id'),
             ],style={"display": "flex", "flex-direction": "row","position": "relative"}),
        ]),
    ]),
   
    html.Div([
    
        
            html.Div([
           
           
           html.Div([
               
               html.H4(children = " 走勢圖"),
                html.Button(id="button1", n_clicks=0,children="切換圖表"),
               dcc.Graph(id="fig_0", figure={})
            ],style={ "flex-direction": "row","position": "relative","left": "2%"}),
           html.Div([
               html.H3(children = "財報"),
               dcc.Graph(figure={}, id='controls-and-graph1')
            ],style={"flex-direction": "row","position": "relative","left": "2%"}), 
           html.Div([
               html.H4(children = " 營收成長率"),
               dcc.Graph(figure={}, id='controls-and-graph2'),
               html.H4(children = "其他財報"),
               dcc.Dropdown(['稅息前利潤','淨利息收入','利息支出','利息收入','淨利','稅前淨利','毛利','成本費用','總收入','營業收入'],'淨利',id='xaxis_column'),
               dcc.Graph(figure={}, id='controls-and-graph3'),
               html.H4(children = "最新新聞"),
               html.Div(id = "word0"),
               html.Div(id = "word1"),
               html.Div(id = "word2"),
               html.Div(id = "word3"),
               html.Div(id = "word4"),
               html.Div(id = "word5"),
               html.Div(id = "word6"),
               html.Div(id = "word7"),
               html.Div(id = "word8"),
               html.Div(id = "word9")
           ],style={"flex-direction": "row","position": "relative","left": "2%"}), 
        
           

       ],className="four columns"),
       
        
   
        
        html.Div([
            html.Div([ 
                html.Div([
                html.Div([dcc.RadioItems([{
                "label": html.Div(['快進快出'], style={'color': 'Gold', 'font-size': 30}),
                "value": "快進快出"
                },{
                "label": html.Div(['長期投資'], style={'color': 'LightGreen', 'font-size': 30}),
                "value": "長期投資"
                },
               
                ], value='長期投資',id='type2', inline=True
                )],style={"display": "flex", "flex-direction": "row","position": "relative"}),
                ]),
                html.Hr(),
                html.H3(id = "word")
            ],className="item_subtitle"),
        ],className="six columns"),
    ])

])

@app.callback(Output("id","children"),
            [Input("button","n_clicks")],
            [State("in","value")]) 
def updafing2(n_clicks ,input_value):
    return id(input_value)

@app.callback(Output("word","children"),
              Output("fig_0","figure", allow_duplicate=True),
              Output(component_id='controls-and-graph1', component_property='figure'),
              Output(component_id='controls-and-graph2', component_property='figure'),
              Output('word0',"children"),
              Output('word1',"children"),
              Output('word2',"children"),
              Output('word3',"children"),
              Output('word4',"children"),
              Output('word5',"children"),
              Output('word6',"children"),
              Output('word7',"children"),
              Output('word8',"children"),
              Output('word9',"children"),
             
              Output(component_id='controls-and-graph3', component_property='figure', allow_duplicate=True),
              Output("loading-output-1","children"),
              [Input("button","n_clicks")],
            [State("in","value")],
            [State("type2","value")],
             prevent_initial_call=True)
def updafig(n_clicks,input_value,type2): 
    s_id = input_value+".TW"
    df = yf.download(s_id,period="10y")
    trace_close = go.Figure(data=[go.Scatter(x=list(df.index),y=list(df.Close))])
    #trace_close = {'data' : [trace_close]}
    trace_close.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
    
    df_stock_fundamental = stock_fundamental(s_id)
    df_stock_fundamental_0=df_stock_fundamental[['每季日期','每季EPS','EPS季增率']]
    df_stock_fundamental_1 = df_stock_fundamental[['每季日期','營收成長率']]
    ff_fig = ff.create_table(df_stock_fundamental_0, height_constant=20,index_title ="財報")
    ff_fig_1 = ff.create_table(df_stock_fundamental_1, height_constant=20,index_title ="財報")
    if (type2 =='長期投資'):
        reply = stock_gpt(s_id,"你是一位長期投資分析股票研究員，口頭禪：只要我們堅持下去，長期一定獲利，一定能取得成功！，『投資就像種樹，需要耐心澆灌。』或『長期來看，市場總是向上的。』，別怕，跟著我，我們一起成為股市偵探！加油;分析報告以長期投資為主")
    if (type2 =='快進快出'):
        reply = stock_gpt(s_id,"你是一位快進快出的交易員，擅長利用技術分析。你的口頭禪是：『機會稍縱即逝，果斷出手！』或『追漲殺跌，方為王道！』請你分析一下最近市場熱門的AI和元宇宙話題。機會來了就抓緊，錯過了就再等等！快進快出，掌握市場節奏！")
    word0 = html.A(f"{news(s_id)[0][1]}", href=f"https://news.cnyes.com/news/id/{www(s_id,0)}")
    word1 = html.A(f"{news(s_id)[1][1]}", href=f"https://news.cnyes.com/news/id/{www(s_id,1)}")
    word2 = html.A(f"{news(s_id)[2][1]}", href=f"https://news.cnyes.com/news/id/{www(s_id,2)}")
    word3=  html.A(f"{news(s_id)[3][1]}", href=f"https://news.cnyes.com/news/id/{www(s_id,3)}")
    word4 = html.A(f"{news(s_id)[4][1]}", href=f"https://news.cnyes.com/news/id/{www(s_id,4)}")
    word5 = html.A(f"{news(s_id)[5][1]}", href=f"https://news.cnyes.com/news/id/{www(s_id,5)}")
    word6 = html.A(f"{news(s_id)[6][1]}", href=f"https://news.cnyes.com/news/id/{www(s_id,6)}")
    word7 = html.A(f"{news(s_id)[7][1]}", href=f"https://news.cnyes.com/news/id/{www(s_id,7)}")
    word8 = html.A(f"{news(s_id)[8][1]}", href=f"https://news.cnyes.com/news/id/{www(s_id,8)}")
    word9 = html.A(f"{news(s_id)[9][1]}", href=f"https://news.cnyes.com/news/id/{www(s_id,9)}")
   
    sleep = ''
    try:
        stock =yf.Ticker(s_id)
        df1 = stock.quarterly_financials
        df1 =df1.iloc[:,0:4]
        df1 = df1.loc[['EBIT','Net Interest Income','Interest Expense','Net Interest Income','Net Income','Pretax Income','Gross Profit','Cost Of Revenue','Total Revenue','Operating Revenue']]
        df1.index=['稅息前利潤','淨利息收入','利息支出','利息收入','淨利','稅前淨利','毛利','成本費用','總收入','營業收入']
        df_stock_fundamental_5=df1.loc[['淨利']]
        df_stock_fundamental_5 =df_stock_fundamental_5.T
        df_stock_fundamental_5=df_stock_fundamental_5.reset_index()
        df_stock_fundamental_5.columns=['每季日期','淨利']
        ff_fig5 = ff.create_table(df_stock_fundamental_5, height_constant=20,index_title ="財報")
    except:
        dfnan = pd.DataFrame({
            "每季日期" : ['nan','nan','nan','nan'],
            "nan" : ['nan','nan','nan','nan'],
           })
        ff_fig5 = ff.create_table(dfnan,height_constant=20,index_title ="財報")


    return reply,trace_close,ff_fig,ff_fig_1,word0,word1,word2,word3,word4,word5,word6,word7,word8,word9,ff_fig5,sleep
@app.callback(Output(component_id='controls-and-graph3', component_property='figure'),
              [Input("xaxis_column","value")],
            [State("in","value")])
def updafig(xaxis_column,input_value): 
    try:
        s_id = input_value+".TW"
        stock =yf.Ticker(s_id)
        df1 = stock.quarterly_financials
        df1 =df1.iloc[:,0:4]
        df1 = df1.loc[['EBIT','Net Interest Income','Interest Expense','Net Interest Income','Net Income','Pretax Income','Gross Profit','Cost Of Revenue','Total Revenue','Operating Revenue']]
        df1.index=['稅息前利潤','淨利息收入','利息支出','利息收入','淨利','稅前淨利','毛利','成本費用','總收入','營業收入']
        df_stock_fundamental_5=df1.loc[[xaxis_column]]
        df_stock_fundamental_5 =df_stock_fundamental_5.T
        df_stock_fundamental_5=df_stock_fundamental_5.reset_index()
        df_stock_fundamental_5.columns=['每季日期',xaxis_column]
        ff_fig5 = ff.create_table(df_stock_fundamental_5, height_constant=20,index_title ="財報")
    except:
        dfnan = pd.DataFrame({
            "每季日期" : ['nan','nan','nan','nan'],
            "nan" : ['nan','nan','nan','nan'],
           })
        ff_fig5 = ff.create_table(dfnan, height_constant=20,index_title ="財報")
    return ff_fig5
@app.callback(Output("fig_0","figure"),
              [Input("button1","n_clicks")],
             [State("in","value")])


def updafig(n_clicks,input_value): 
    s_id = input_value+".TW"
    df = yf.download(s_id,period="10y")
    if n_clicks%2==0:
        trace_close = go.Figure(data=[go.Scatter(x=list(df.index),y=list(df.Close))])
        #trace_close = {'data' : [trace_close]}
        trace_close.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
        
    else:
        trace_close= go.Figure(data=[go.Candlestick(x=df.index,
                                   open=df['Open'],
                                   high=df['High'],
                                   low=df['Low'],
                                   close=df['Close'],
                                   increasing_line_color='red',
                                   decreasing_line_color='green')])
        trace_close.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
    return trace_close
    

    

if __name__ =="__main__":
    app.run_server(debug=True)

[*********************100%%**********************]  1 of 1 completed


---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
Cell In[2], line 232, in updafig(n_clicks=0, input_value=None)
    226 @app.callback(Output("fig_0","figure"),
    227               [Input("button1","n_clicks")],
    228              [State("in","value")])
    229 
    230 
    231 def updafig(n_clicks,input_value): 
--> 232     s_id = input_value+".TW"
        input_value = None
    233     df = yf.download(s_id,period="10y")
    234     if n_clicks%2==0:

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
Cell In[2], line 232, in updafig(n_clicks=0, input_value=None)
    226 @app.callback(Output("fig_0","figure"),
    227               [Input("button1","n_clicks")],
    228              [State("in","value")])
    229

[*********************100%%**********************]  1 of 1 completed
C:\Users\user\AppData\Local\Temp\ipykernel_21592\2250803348.py:12: FutureWarning:

The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.

C:\Users\user\AppData\Local\Temp\ipykernel_21592\2250803348.py:12: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

C:\Users\user\AppData\Local\Temp\ipykernel_21592\2250803348.py:16: FutureWarning:

The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None

請依據以下資料來進行分析並給出一份完整的分析報告:
每季營收資訊：
         每季日期  營收成長率  每季EPS  EPS季增率
0 2024-09-30  -0.05   1.70   -0.22
1 2024-06-30  -0.04   2.19   -0.18
2 2024-03-31  -0.37   2.66    0.00
3 2023-12-31   0.37    NaN    1.02
近期新聞資訊: 
 [['2024-12-04', '盤中速報 - 世紀鋼(<mark>9958</mark>)大漲7.18%，報194元'], ['2024-11-19', '世紀鋼:公告本公司發言人異動'], ['2024-11-15', '盤中速報 - 世紀鋼(<mark>9958</mark>)急拉3.07%報184.5元，成交1,640張'], ['2024-11-14', '《神州民企》力天影業(0<mark>9958</mark>)與山西寧樂或組合資,訂備忘'], ['2024-11-12', '世紀鋼:公告本公司董事會決議現金增資子公司世紀離岸風電設備(股)公司(董事會決議提高交易數量及交易總金額)'], ['2024-11-12', '世紀鋼:本公司董事會通過113年度第三季合併財務報告'], ['2024-11-08', '盤中速報 - 世紀鋼(<mark>9958</mark>)大跌7.21%，報186.5元'], ['2024-11-07', '世紀鋼:世紀鋼鐵結構股份有限公司國內第八次無擔保轉換公司債上櫃公告'], ['2024-11-06', '《財資快訊》美電軟報7.7735,隔夜拆息與上日尾市持平'], ['2024-11-06', '世紀鋼:公告本公司發行國內第八次無擔保轉換公司債募集完成']]
 做出近期的趨勢報告,請以詳細、      嚴謹及專業的角度撰寫此報告,並提及重要的數字, reply in 繁體中文


C:\Users\user\AppData\Local\Temp\ipykernel_21592\2250803348.py:42: DeprecationWarning:

datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).



---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
Cell In[2], line 232, in updafig(n_clicks=0, input_value=None)
    226 @app.callback(Output("fig_0","figure"),
    227               [Input("button1","n_clicks")],
    228              [State("in","value")])
    229 
    230 
    231 def updafig(n_clicks,input_value): 
--> 232     s_id = input_value+".TW"
        input_value = None
    233     df = yf.download(s_id,period="10y")
    234     if n_clicks%2==0:

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
Cell In[2], line 232, in updafig(n_clicks=0, input_value=None)
    226 @app.callback(Output("fig_0","figure"),
    227               [Input("button1","n_clicks")],
    228              [State("in","value")])
    229